In [ ]:
import os,json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import keras
from keras.datasets import reuters
import gensim
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Conv1D, MaxPooling1D, Dropout
from keras.utils.np_utils import to_categorical


In [4]:
# this finds our json files
path_to_json = 'D:\Career\Spotmentor Technologies\mohitatbb-machine-learning-assessment-2ef59f0cd1ce\data\docs'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files)  

['5922081.json', '6523883.json', '6600981.json', '6686185.json', '6726409.json', '6975593.json', '6978067.json', '6978079.json', '7000953.json', '7010467.json', '7045929.json', '7072743.json', '7093373.json', '7121587.json', '7128611.json', '7135099.json', '7135571.json', '7140501.json', '7160943.json', '7172353.json', '7238217.json', '7291285.json', '7315159.json', '7424001.json', '7424203.json', '7424353.json', '7465053.json', '7494221.json', '7494229.json', '7521301.json', '7521649.json', '7527075.json', '7533733.json', '7534839.json', '7540071.json', '7549735.json', '7549739.json', '7590259.json', '7622571.json', '7636811.json', '7641863.json', '7654483.json', '7655893.json', '7667349.json', '7674559.json', '7709455.json', '7710189.json', '7719949.json', '7719991.json', '7754159.json', '7754317.json', '7757525.json', '7760163.json', '7761203.json', '7770319.json', '7776307.json', '7792815.json', '7794234.json', '7794524.json', '7795949.json', '7795967.json', '7796107.json', '779738

In [5]:
# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['Description','ID'])
#jsons_data

In [6]:
# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        
# here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        Description = json_text['jd_information']['description']
        ID = json_text['_id']
        #lonlat = json_text['features'][0]['geometry']['coordinates']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data.loc[index] = [Description,ID]

# now that we have the pertinent json data in our DataFrame let's look at it
print(jsons_data)

                                            Description       ID
0     Urgent positions for Travel Executive at Saane...  5922081
1     Dear Candidate,Greetings From Continental Immi...  6523883
2     We Have urgent poitions for Ticketing Executiv...  6600981
3     We Have Urgent positions for Travel booking cu...  6686185
4     Experience in Travel Industry.We have urgent o...  6726409
5     We have an urgent opening for&nbsp;Customer ca...  6975593
6     Need Urgent for Male candidates for sales Exec...  6978067
7     Need Urgent for Marketing Head Cum Corporate c...  6978079
8     We have an urgent opening for Female Personal ...  7000953
9     Urgent opening for Air Ticking &amp; tour pack...  7010467
10    Hello Readers,UTS Is An ISO Certified Entity L...  7045929
11    Dear Candidates,Greetings From Unique Travelit...  7072743
12    Greetings From Unique Tourism services!Unique ...  7093373
13    Urgent Positions required for Corporate Sales ...  7121587
14    Hello to all reader

In [7]:
jsons_data.shape

(1162, 2)

In [8]:
#reading document_departments.csv
dept = pd.read_csv('D:\Career\Spotmentor Technologies\mohitatbb-machine-learning-assessment-2ef59f0cd1ce\data\document_departments.csv')

In [9]:
dept.head()

,Document ID,Department
0,8196431,Customer service
1,8278123,Digital Marketing
2,8331625,Sales
3,8331003,Back office ticketing
4,8277939,Digital Marketing


In [10]:
#renaming Document ID column in order to merge both dataframes on 'ID'
dept=dept.rename({'Document ID':'ID'},axis = 1)
dept.head()

,ID,Department
0,8196431,Customer service
1,8278123,Digital Marketing
2,8331625,Sales
3,8331003,Back office ticketing
4,8277939,Digital Marketing


In [11]:
dept.dtypes
#ID type is int64

ID             int64
Department    object
dtype: object

In [12]:
jsons_data.dtypes
#ID type is object

Description    object
ID             object
dtype: object

In [13]:
#coverting jsons_data's ID type from object to int to merge dept and jsons_data
jsons_data = jsons_data.astype({"ID": int})
jsons_data.dtypes

Description    object
ID              int32
dtype: object

In [14]:
#merge both dataframes on ID
model_data = pd.merge(jsons_data, dept,  how = 'left', on=['ID'])

In [15]:
model_data.head()

,Description,ID,Department
0,Urgent positions for Travel Executive at Saane...,5922081,Customer service
1,"Dear Candidate,Greetings From Continental Immi...",6523883,Sales
2,We Have urgent poitions for Ticketing Executiv...,6600981,Ticketing
3,We Have Urgent positions for Travel booking cu...,6686185,Customer service
4,Experience in Travel Industry.We have urgent o...,6726409,Ticketing


In [16]:
model_data.shape

(1162, 3)

In [17]:
#data preprocessing
#data cleaning
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
model_data['Description'] = model_data['Description'].apply(clean_text)
model_data['Description'].head()

0    urgent positions travel executive saanewaal lo...
1    dear candidate greetings continental immigrati...
2    urgent poitions ticketing executive tour trave...
3    urgent positions travel booking cum client sup...
4    experience travel industrywe urgent opening fo...
Name: Description, dtype: object

In [18]:
model_data['Description'].apply(lambda x: len(x.split(' '))).sum()
#After text cleaning and removing stop words, we have only over 50k words to work with!

52680

In [19]:
#converting 'Department' column to list in order to find unique elements in it for label encoding
dept_list=model_data['Department'].tolist()
dept_list

['Customer service',
 'Sales',
 'Ticketing',
 'Customer service',
 'Ticketing',
 'Ticketing',
 'Sales',
 'Ticketing',
 'Administration',
 'Ticketing',
 'Digital Marketing',
 'Public Relations ',
 'Digital Marketing',
 'Sales',
 'Marketing',
 'IT',
 'Marketing',
 'Digital Marketing',
 'Marketing',
 'Ticketing',
 'Ticketing',
 'Public Relations ',
 'Sales',
 'Ticketing',
 'Ticketing',
 'Sales',
 'Sales',
 'Ticketing',
 'Ticketing',
 'Administration',
 'Sales',
 'Sales',
 'Back office ticketing',
 'Sales',
 'Sales',
 'Maintenance',
 'Maintenance',
 'Customer service',
 'Ticketing',
 'Sales',
 'Sales',
 'Sales',
 'Sales',
 'Digital Marketing',
 'Digital Marketing',
 'Digital Marketing',
 'Ticketing',
 'Technology',
 'Sales',
 'Ticketing',
 'Finance',
 'Ticketing',
 'Ticketing',
 'Ticketing',
 'Ticketing',
 'Ticketing',
 'Ticketing',
 'Sales',
 'Ticketing',
 'Ticketing',
 'Finance',
 'Learning and Development ',
 'Ticketing',
 'Technology',
 'Sales',
 'Sales',
 'Ticketing',
 'Ticketing',
 '

In [22]:
#creating a lookup table for reference of department names vs it's corresponding labels
dept_uniq=np.unique(dept_list)
dept_uniq
dept_label=label_encoder.fit_transform(dept_uniq) 
#model_data['Department'].unique()
dept_label
lookup = pd.DataFrame({'dept_uniq':dept_uniq, 'dept_label':dept_label})
lookup


,dept_uniq,dept_label
0,Administration,0
1,Airline Ground Operations,1
2,Analytics,2
3,Back office ticketing,3
4,Content,4
5,Customer service,5
6,Data entry,6
7,Digital Marketing,7
8,Engineering Design Construction,8
9,Finance,9


In [23]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
model_data['Department']= label_encoder.fit_transform(model_data['Department']) 
  
model_data['Department'].unique() 
model_data.head()

,Description,ID,Department
0,urgent positions travel executive saanewaal lo...,5922081,5
1,dear candidate greetings continental immigrati...,6523883,27
2,urgent poitions ticketing executive tour trave...,6600981,29
3,urgent positions travel booking cum client sup...,6686185,5
4,experience travel industrywe urgent opening fo...,6726409,29


In [24]:
#tokenizing out description feature for applying word2vec model on 'Description' feature
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

des=list()
lines=model_data['Description'].values.tolist()

for line in lines:
    tokens=word_tokenize(line)
    #remove tokens that are not alphabetic
    words=[word for word in tokens if word.isalpha()]
    #filter out syop words
    stop_words=set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    des.append(words)

In [25]:
len(des)

1162

In [26]:
#train word2vec model
model = gensim.models.Word2Vec(sentences=des, size= 100, window=5, workers=4, min_count=1)
#vocab size
words=list(model.wv.vocab)
len(words)


7076

In [27]:
#save model
filename= 'des_word2vec.txt'
model.wv.save_word2vec_format(filename)

In [28]:
#use pretrained embeddings
emb_ind={}
f=open(os.path.join('', 'des_word2vec.txt'),encoding = 'utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:])
    emb_ind[word]=coefs
f.close()

In [29]:
#vectorize the text samples into 2D tensor
tokenizer = Tokenizer()
tokenizer.fit_on_texts(des)
sequences = tokenizer.texts_to_sequences(des)

#pad sequences
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

des_pad=pad_sequences(sequences,maxlen=1000)
Dept=model_data['Department'].values

print(des_pad.shape)
print(Dept.shape)


Found 7076 unique tokens.
(1162, 1000)
(1162,)


In [30]:
#Now we will map embeddings from the loaded word2vec model for each word to the 
#tokenizer_obj.word_index vocabulary and create a matrix with of word vectors.
num_words=len(word_index)+1
emb_matrix=np.zeros((num_words,100))

for word, i in word_index.items():
    if i>num_words:
        continue
    emb_vector=emb_ind.get(word)
    if emb_vector is not None:
        #words not found in emb_ind will be all zeros
        emb_matrix[i]=emb_vector
print(num_words)
#len(emb_matrix[0])

7077


In [31]:
#We are now ready with the trained embedding vector to be used directly in the embedding layer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

#define model
Embed_Dim=100
max_len=1000
model=Sequential()
embedding_layer=Embedding(num_words,Embed_Dim, embeddings_initializer=Constant(emb_matrix),
                           input_length=max_len,trainable=False)
model.add(embedding_layer)
model.add(Conv1D(300, 3, padding='valid', activation='relu', strides=1))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dense(64, activation='relu')) #try from 64,128,256,512. O?P layer=Relu. A.9.
#model.add(Dropout(0.5))
#model.add(Dense(32, activation='relu', input_dim=(1162,100)))
model.add(Dense(30, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         707700    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 998, 300)          90300     
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                31968     
_________________________________________________________________
dense_1 (Dense)              (None, 30)                990       
Total params: 830,958
Trainable params: 123,258
Non-trainable params: 707,700
_________________________________________________________________


In [32]:
#split data into training and validation set

test_split=0.2
indices=np.arange(des_pad.shape[0])
#np.random.shuffle(indices)
np.random.seed(46)
des_pad=des_pad[indices]
num_test_samples=int(test_split*des_pad.shape[0])
X_train_pad = des_pad[:-num_test_samples]
y_train=Dept[:-num_test_samples]
X_test_pad=des_pad[-num_test_samples:]
y_test=Dept[-num_test_samples:]

In [33]:
print(X_train_pad.shape)
print(y_train.shape)
print(X_test_pad.shape)
print(y_test.shape)

(930, 1000)
(930,)
(232, 1000)
(232,)


In [35]:
#Finally training the classification model on train and validation test set
model.fit(X_train_pad, y_train, batch_size=128, epochs=50, validation_data=(X_test_pad,y_test),verbose=2)
#accuracy achieved here is almost 56%

Train on 930 samples, validate on 232 samples
Epoch 1/50
 - 18s - loss: 1.6575 - acc: 0.4860 - val_loss: 2.4274 - val_acc: 0.2500
Epoch 2/50
 - 18s - loss: 1.6428 - acc: 0.4957 - val_loss: 2.4184 - val_acc: 0.2500
Epoch 3/50
 - 19s - loss: 1.6410 - acc: 0.5022 - val_loss: 2.4118 - val_acc: 0.2500
Epoch 4/50
 - 19s - loss: 1.6476 - acc: 0.4946 - val_loss: 2.4157 - val_acc: 0.2888
Epoch 5/50
 - 19s - loss: 1.6355 - acc: 0.4774 - val_loss: 2.4047 - val_acc: 0.2802
Epoch 6/50
 - 20s - loss: 1.6307 - acc: 0.5011 - val_loss: 2.4239 - val_acc: 0.2500
Epoch 7/50
 - 22s - loss: 1.6208 - acc: 0.5108 - val_loss: 2.4147 - val_acc: 0.2543
Epoch 8/50
 - 22s - loss: 1.6190 - acc: 0.5140 - val_loss: 2.4168 - val_acc: 0.2543
Epoch 9/50
 - 21s - loss: 1.6250 - acc: 0.5247 - val_loss: 2.4271 - val_acc: 0.2328
Epoch 10/50
 - 20s - loss: 1.6208 - acc: 0.5043 - val_loss: 2.4190 - val_acc: 0.2759
Epoch 11/50
 - 21s - loss: 1.6105 - acc: 0.5032 - val_loss: 2.4247 - val_acc: 0.2759
Epoch 12/50
 - 20s - loss: 1